In [3]:
import pandas as pd
import re
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as stp
from sklearn.feature_extraction.text import CountVectorizer
# nltk.download()

In [4]:
df = pd.read_csv('Resume Scanner.csv')
df_cp = df.copy()

C:\Users\shish\AppData\Local\Temp\ipykernel_24780\4279448597.py:1: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Resume Scanner.csv')


In [5]:
df.head()

,accomplishments_segment,degree,education_segment,emails,job_titles,links,misc_segment,name,objectives_segment,phone,...,Unnamed: 185,Unnamed: 186,Unnamed: 187,Unnamed: 188,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194
0,NaN,"MS , B.TECH , PhD",NaN,sutarvinayak2063@gmail.com,"full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...",NaN,Prasad Nagar,NaN,9.19309E+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SUTAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LinkedIn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,+91 9309161108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sutarvinayak2063@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.isnull().sum()


accomplishments_segment      2122
degree                     159447
education_segment          183175
emails                     193963
job_titles                 199370
                            ...  
Unnamed: 190               210371
Unnamed: 191               210371
Unnamed: 192               210371
Unnamed: 193               210371
Unnamed: 194               210371
Length: 195, dtype: int64

In [7]:
df.drop(['accomplishments_segment','education_segment','emails','misc_segment','name','objectives_segment','phone','projects_segment','skills_segment','text',
         'university_0','university_1','university_2','university_3','university_4','university_5','url','work_segment'],axis=1,inplace=True)
df.shape

(210372, 177)

In [8]:
print('After removing the columns from dataset..')
df.isnull().sum()

After removing the columns from dataset..


degree             159447
job_titles         199370
links              203160
skills             208612
work_experience    209988
                    ...  
Unnamed: 190       210371
Unnamed: 191       210371
Unnamed: 192       210371
Unnamed: 193       210371
Unnamed: 194       210371
Length: 177, dtype: int64

In [9]:
df['degree'].fillna(df.degree.mode()[0],inplace=True)
df['links'].fillna('Missing',inplace=True)
df['work_experience'].fillna(0,inplace=True)
df['job_titles'].fillna('No Job',inplace=True)

In [10]:
all_degrees = ''
for i in df.degree:
  if len(all_degrees) == 0:
    all_degrees = i
  else:
    all_degrees = all_degrees + ' , ' + i

all_degrees = all_degrees.split(',')
all_degrees = [re.sub("[\s.]","",i).upper() for i in all_degrees]
unique_degrees = set(all_degrees)
unique_degrees

{'',
 'JBOSSANDTOMCAT',
 'METINTERNALANDEXTERNALOBJECTIVES:',
 'CSS3ANDREACTJS',
 '&SOON;ENABLINGITINFRASTRUCTUREASFULLYAUTOMATED',
 'IREPORTRE-WRITE',
 'COMMUNICATIONTEMPLATES',
 'CREATEIMAGEOFTHESYSTEM',
 'CATEGORYMANAGEMENTANDRETAILMANAGEMENT(OFFLINE&ONLINE)RECOGNIZEDASA',
 'AGILITY',
 'IOT(THINGWORX)',
 '2',
 'SOCIAL',
 'TAS',
 'ANALYZINGDEFECTSANDDEVELOPINGPATCHES',
 'CAMPUS',
 '09/02/2021HTTPS://FREESEARCHNAUKRICOM/PREVIEW/PRINTRESUME?UNAME=B3710D9B798EF218269AF1D1F309644B0C5400524E1B084004134309580C031F465D14…',
 'OFFICIALTECHNO-MANAGEMENTFESTOFNITSILCHARTHISAPPCONSISTSOFREALTIME',
 'ALONGWITHREDUCINGOPERATIONALCOSTONAWSTOOLSAND',
 'INTERNATIONALBUSINESS',
 'IPA',
 'EXECUTION&IMPLEMENTATIONTEAMSIZE:5',
 'DURGIN',
 'ALAPPUZHA',
 'PROJECTCOST',
 'PROMETHEUS',
 'CANARYDEPLOYMENTSANDIN-PLACEDEPLOYMENTSUSINGAWSCODEDEPLOY',
 'CREATINGS3BUCKET',
 'ANDSWITCHES',
 'SHAPE',
 'LAYOUTSKETCHES',
 'SAHSON',
 'ERWINDATAMODELER',
 'E-COMMERCEANDIOTTEAMSANDRESOLVINGDEPENDENCIESFORTHESAMEAND',
 '

In [11]:
df['bachelor_degrees'] = 'No Degree'
df['master_degrees'] = 'No Degree'
df['docterte_degrees'] = 'No Degree'
df['profiles'] = 'No Profile'

In [12]:
ind = 0
for i in df.degree:  
  lst = re.sub("[\s.]","",i).upper( )
  # print(lst)
  for j in lst.split(','):
    if j in ['BE', 'BS', 'BSC', 'BTECH']:
      if df.loc[ind,'bachelor_degrees'] == 'No Degree':
        df.loc[ind,'bachelor_degrees'] = j
      else:
        df.loc[ind,'bachelor_degrees'] = df.loc[ind,'bachelor_degrees'] + ' , '+ j
    elif j in ['ME', 'MS', 'MSC', 'MTECH']:      
      if df.loc[ind,'master_degrees'] == 'No Degree':
        df.loc[ind,'master_degrees'] = j
      else:
        df.loc[ind,'master_degrees'] = df.loc[ind,'master_degrees'] + ' , '+ j
    elif j == 'PHD':      
      if df.loc[ind,'docterte_degrees'] == 'No Degree':
        df.loc[ind,'docterte_degrees'] = j
      else:
        df.loc[ind,'docterte_degrees'] = df.loc[ind,'docterte_degrees'] + ' , '+ j
  ind +=1

In [13]:
ind = 0
for i in df.links:
  lst = re.sub("[\s]","",i)
  # print(lst)
  for j in lst.split(','):
    if j.find('github') != -1:          
      df.loc[ind,'profiles'] = 'Github'
    elif j.find('linkedin') != -1:
      if df.loc[ind,'profiles'] == 'No Profile':
        df.loc[ind,'profiles'] = 'Linkedin'
      else:
        df.loc[ind,'profiles'] = df.loc[ind,'profiles'] + ' , ' + 'Linkedin'
  ind +=1

In [14]:
df

,degree,job_titles,links,skills,work_experience,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,...,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BTECH,MS,PHD,Github
1,Online,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile
2,Online,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile
3,Online,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile
4,Online,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210367,Online,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile
210368,Online,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile
210369,Online,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile
210370,,No Job,Missing,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Degree,No Degree,No Degree,No Profile


In [15]:
# list(df['work_experience'].unique())
df.work_experience.value_counts()

0                  209988
 english               14
 ltd                   12
 hindi                 12
 designation           12
                    ...  
 loans                  1
 juniors                1
 ossec                  1
 life sciences          1
 artifactory            1
Name: work_experience, Length: 234, dtype: int64

In [16]:
ind = 0
i=0
for i in df.work_experience:
  if i < 0:
    df.loc[ind,'work_experience'] = i*-1
  elif i > 40:
    df.loc[ind,'work_experience'] = 0
  ind +=1
df['work_experience'] = df['work_experience'].astype(int)

TypeError: '<' not supported between instances of 'str' and 'int'